In [ ]:
! pip install trubrics

## Load data & model

In [ ]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()

## Init DataContext

In [ ]:
from trubrics.context import DataContext

In [ ]:
data_context = DataContext(
    name="my_first_dataset",
    version="0.0.1",
    target="Survived",
    training_data=train_df,
    testing_data=test_df,
)

## Init ModelValidator

In [ ]:
from trubrics.validations import ModelValidator
model_validator = ModelValidator(data=data_context, model=model)

## Build validations

In [ ]:
import rich # pretty print

In [ ]:
performance_validations = [
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.8, severity="experiment"),
    model_validator.validate_test_performance_against_dummy(metric="accuracy", strategy="constant", dummy_kwargs={"constant": 0}),
]
for performance_val in performance_validations:
    rich.print(performance_val)

Check out the docs for many more [out-of-the-box validations](https://trubrics.github.io/trubrics-sdk/validations/) and [custom validations](https://trubrics.github.io/trubrics-sdk/custom_validations/).

## Group validations in a trubric

In [ ]:
from trubrics.validations import Trubric

trubric = Trubric(
    name="classification_trubric",
    model_name="my_model",
    model_version="0.0.1",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    tags=["nb-demo"],
    validations=performance_validations,
)

## Save Locally

In [ ]:
# save trubric to a local .json
trubric.save_local(path="my_first_trubric.json")

## Save trubric to the Trubrics platform

First, we run `trubrics init` to connect to the platform:

In [ ]:
import os
import getpass
os.environ["TRUBRICS_CONFIG_EMAIL"] = input("Enter your Trubrics account email:")
os.environ["TRUBRICS_CONFIG_PASSWORD"] = getpass.getpass("Enter your password:")
os.environ["TRUBRICS_PROJECT_NAME"] = input("Enter your Trubrics project name:")

In [ ]:
! trubrics init --trubrics-user --project-name "$TRUBRICS_PROJECT_NAME"

Now we can save our `trubric` (or `new_trubric`) with the save_ui() method directly from our notebook, or a python script:

In [ ]:
try:
    trubric.save_ui()
except Exception as e:
    print(f"Error in saving to the Trubrics platform:\n{e}")

or we can save it within an automated pipeline with the CLI:

In [ ]:
! trubrics run \
  --save-ui \
  --run-context-path titanic-example-trubric \
  --trubric-output-file-path "cli_demo_trubric.json"

Note: this demo of our CLI will run our example trubric, not the trubric that we have saved in this tutorial.